### 导入相关库并设置所需全局变量

1. 导入运行本项目所需的库。

2. 设置device全局变量，指定模型和张量的计算设备，判断设备是否支持CUDA并使用GPU进行加速计算，否者使用CPU。

In [1]:
import re
from collections import Counter
import torch
from torch import nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# torch.device('cpu/cuda/cuda:device_index')指定模型和张量的计算设备
# 注意模型和张量的设备一致性，在进行训练或推理时，需确保模型和张量在同一个设备上
# torch.cuda.is_available()检查当前系统是否支持CUDA，是否有可用GPU设备，判断当前环境是否可使用GPU进行加速计算

### 定义并封装文本和字符串的预处理方法

使用类封装被@staticmethod装饰器装饰的各种文本或字符串的预处理方法。

@staticmethod装饰器用于定义静态方法，静态方法将不依赖类实例，也不需访问类或实例的任何属性或方法，可通过类名直接调用，也可通过类实例调用。

1. 文本预处理方法textYuChuLi(text)，接收文件地址，读取文件并转为字符串类型，使用正则表达式将所有非字母和连续空格转为单个空格，将大写转为小写，返回字符串。

2. 字符串预处理方法stringYuChuLi(string)，接收字符串，使用正则表达式将所有非字母和连续空格转为单个空格，将大写转为小写，返回字符串。

3. 统计字符串中空格和各种单词数量方法conunt(string)，接收字符串，使用正则表达式统计空格数量，使用分割和统计接口类统计各种单词的数量，打印输出。

4. 构建双向字典词表方法vocabulary(string)，接收字符串，构建两个字典，遍历字符串记录每种字符，一个是键为字符，值为数字，一个是键为数字，值为字符，返回两个字典。

5. 字符串转数字序列方法charToIndexTransform(string, charToIndex)，接收字符串和字符转数字字典词表，通过列表和遍历返回数字序列列表。

6. 数字序列转字符序列方法indexToCharTransform(index, indexToChar)，接收数字序列列表和数字转字符字典词表，通过列表和遍历返回字符序列列表。

7. 截取序列获得样本方法example(sequence, window)，接收序列列表和指定截取窗口大小，来截取序列，获得训练样本，包括特征和标签。

注意：

- 为达到完美预处理效果，要注意预处理方法的先后顺序。

- 为达到完美预处理效果，要注意正则表达式的先后顺序。

- 统计字符串中空格和各种单词数量方法打印的结果可能较多，对于长序列数据，如果绘图统计，那么各种单词的数量很可能以一种明确的方式迅速衰减，满足齐普定律，即很多单词的出现数量很少，应当采用适当方法处理。

- 构建双向字典词表用于字符串转为数字序列和数字序列转为字符序列，便于模型的计算操作和预测数字序列的转换。

- 以上方法通用性较好，可加以改造适用其他情况。

In [2]:
class YuChuLi:
    # 文本预处理
    @staticmethod
    # @staticmethod装饰器，定义静态方法，类的特殊方法，不依赖类实例，也不需访问类或实例的任何属性或方法，可通过类名直接调用，也可通过类实例调用
    def textYuChuLi(text):
        with open(text) as text:
        # with语句简化资源管理操作，主要作用是与上下文管理器一起使用，确保资源在使用完毕后能被正确释放
        # open()打开文件并返回文件对象，通过文件对象，可对文件进行读写操作
            text = text.read()
            # read()文件对象方法，从文件中读取数据，可一次性读取整个文件内容，或按指定字节数读取部分内容，返回字符串或字节对象，表示从文件中读取的数据
 
            # 正则表达式匹配所有非字母和非空格的字符转为空格
            text = re.sub(r'[^a-zA-Z\s]', ' ', text)
            # 替换连续空格为单个空格
            text = re.sub(r'\s+', ' ', text)
 
            text = text.lower()
            # lower()字符串对象方法，将字符串中所有大写字母转为小写字母，返回新字符串
        return text
 
    # 字符串预处理
    @staticmethod
    def stringYuChuLi(string):
        string = re.sub(r'[^a-zA-Z\s]', ' ', string)
        string = re.sub(r'\s+', ' ', string)
        string = string.lower()
        return string
    
    # 统计字符串中空格和各种单词数量
    @staticmethod
    def conunt(string):
        # 正则表达式匹配所有空格并统计
        spacesCount = len(re.findall(r'\s', string))
 
        # 统计各种单词数量
        string = string.split()
        # split()字符串对象方法，将字符串分割成多个子字符串，返回包含这些子字符串的列表，基于指定分隔符进行分割，默认使用空格
        wordCount = Counter(string)
        # Counter类，计数可哈希对象，提供接口统计元素出现次数，并提供方法处理计数，返回Counter对象，字典子类，键是输入数据元素，值是元素出现次数
        wordCount = wordCount.most_common()
        # most_common()类Counter的方法，返回输入数据中出现次数最多的k个元素及其计数，可选，默认为10
        # 返回列表，每个元素都是元组，第一个元素是输入数据，第二个元素是元素出现次数，列表按照元素出现次数的降序排列
 
        print(f'空格：{spacesCount}个')
        for word, count in wordCount:
            print(f'{word}：{count}个')
        
    # 构建双向字典词表
    @staticmethod
    def vocabulary(string):
        charToIndexVocabulary = dict()
        indexToCharVocabulary = dict()
        index = 0
        for char in string:
            if char not in charToIndexVocabulary.keys():
                charToIndexVocabulary[char] = index
                indexToCharVocabulary[index] = char
                index += 1
        return charToIndexVocabulary, indexToCharVocabulary
    
    # 字符序列串转数字序列
    @staticmethod
    def charToIndexTransform(string, charToIndex):
        indexSequence = []
        for char in string:
            if char in charToIndex:
                indexSequence.append(charToIndex[char])
        return indexSequence
 
    # 数字序列转字符序列
    @staticmethod
    def indexToCharTransform(index, indexToChar):
        charSequence = []
        for index in index:
            if index in indexToChar:
                charSequence.append(indexToChar[index])
        return charSequence
 
    # 截取序列获得样本
    @staticmethod
    def example(sequence, window):
        feature, label= [], []
        for i in range(len(sequence) - window  - 1):
            feature.append(sequence[i : i + window])
            label.append(sequence[i + window + 1])
        return feature, label

In [3]:
# 相关函数/方法


# with expression as variable:
# expression通常是调用上下文管理器协议的对象
# variable可选，expression.__enter__()的返回值会赋值给此变量，在代码块内部可使用
 
 
# open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
# file要打开的文件的路径
# mode文件打开模式，默认为'r'
#   r只读模式（默认）
#   w写入模式，如果文件存在则覆盖，不存在则创建
#   a追加模式，如果文件存在则在末尾追加，不存在则创建
#   x独占创建模式，如果文件已存在则报错
#   b二进制模式
#   t文本模式（默认）
#   +更新模式，可以读写
# buffering设置缓冲策略，默认为-1，系统默认缓冲
# encoding文件编码格式，仅在文本模式下有效
# errors如何处理编码和解码错误，默认为None
# newline控制换行符的处理方式，仅在文本模式下有效
# closefd如果为True，则在文件关闭时关闭底层文件描述符，默认为True
# opener自定义打开器，替代内置的打开机制
 
 
# file.read(size=-1)
# size指定要读取的字节数，如果省略或设置为-1，则读取整个文件的内容
# 每次调用read()方法后，文件指针会移动到读取内容的末尾，如果需要重新读取文件，可使用file.seek(0)将文件指针重置到文件开头
# 对于大文件，一次性读取整个文件可能会导致内存不足，可使用file.readline()或file.readlines()逐行读取文件或使用迭代器逐块读取文件内容
 
 
# str.split(sep=None, maxsplit=-1)
# sep指定分隔符，如果省略或设为None，则默认使用空格作为分隔符
# maxsplit指定最大分割次数，如果设为正整数，则最多分割maxsplit次，如果设为-1，默认，则分割所有匹配的分隔符
 
 
# counter = Counter([iterable-or-mapping])
# iterable-or-mapping可迭代对象或映射对象，如果是可迭代对象，会统计每个元素的出现次数，如果是映射对象，会使用映射对象的键值对来初始化计数
 
 
# most_common()返回列表，每个元素都是元组，第一个元素是输入数据中的元素，第二个元素是该元素在输入数据中出现的次数，列表按照元素出现次数的降序排列

### 文本预处理

应用文本和字符串的预处理类的方法处理文本并获得训练模型使用的样本

1. 文本预处理并打印统计字符串中空格和各种单词的数量

In [4]:
text = YuChuLi.textYuChuLi('C:\\Users\\kongbai\\study\\dataset\\book.txt')
count = YuChuLi.conunt(text)

空格：5991个
the：312个
and：267个
to：180个
i：139个
a：129个
of：120个
that：102个
you：83个
it：77个
in：68个
we：63个
hugh：62个
was：59个
they：57个
s：56个
there：52个
he：49个
t：47个
jack：45个
said：45个
had：43个
for：43个
all：41个
be：40个
on：38个
his：34个
but：33个
out：30个
were：30个
as：29个
have：29个
so：29个
this：28个
at：28个
him：27个
one：27个
them：27个
up：26个
with：25个
ve：25个
good：24个
are：24个
over：23个
well：23个
got：23个
would：23个
down：22个
when：22个
d：22个
ll：22个
two：22个
then：21个
see：21个
from：21个
can：21个
here：21个
time：21个
been：21个
just：21个
now：20个
right：20个
or：20个
about：19个
get：19个
horses：18个
re：18个
birds：18个
don：17个
again：17个
before：17个
what：17个
by：17个
pack：16个
their：16个
long：15个
off：15个
not：15个
know：15个
come：15个
back：15个
your：15个
while：15个
some：15个
if：15个
do：15个
go：14个
ranch：14个
make：14个
pretty：14个
em：14个
after：14个
which：14个
me：13个
these：13个
put：12个
three：12个
expect：12个
did：12个
winter：12个
wolf：12个
skin：12个
more：11个
horse：11个
than：11个
thought：11个
much：11个
how：11个
mighty：11个
few：11个
little：11个
lot：11个
will：11个
along：11个
cow：11个
like：11个
othe

1. 构建双向字典词表并打印。

2. 将字符串化的文本数据进行字符串转数字序列，然后初始化截取窗口大小，进行序列截取，获得训练样本，包括特征和标签。

In [5]:
charToIndexVocabulary, indexToCharVocabulary = YuChuLi.vocabulary(text)
for key, value in charToIndexVocabulary.items():
    print(f"{key}: {value}")
for key, value in indexToCharVocabulary.items():
    print(f"{key}: {value}")
textIndexSequence = YuChuLi.charToIndexTransform(text, charToIndexVocabulary)
window = 5
feature, label = YuChuLi.example(textIndexSequence, window)

t: 0
h: 1
e: 2
 : 3
r: 4
a: 5
i: 6
n: 7
u: 8
s: 9
d: 10
o: 11
w: 12
l: 13
g: 14
k: 15
b: 16
m: 17
y: 18
j: 19
c: 20
f: 21
p: 22
v: 23
z: 24
x: 25
q: 26
0: t
1: h
2: e
3:  
4: r
5: a
6: i
7: n
8: u
9: s
10: d
11: o
12: w
13: l
14: g
15: k
16: b
17: m
18: y
19: j
20: c
21: f
22: p
23: v
24: z
25: x
26: q


### 文本预处理结果向量化并创建数据迭代器

1. 将样本的特征和标签数据转为张量，根据设备硬件情况，可将张量移动到指定设备上进行计算。

2. 将特征和标签张量组合成一个数据集，指定批量大小和是否在每个epoch开始时打乱数据集，创建数据迭代器，便于模型在训练过程中循环迭代。

In [6]:
featureTensor = torch.tensor(feature, device=device)
# torch.tensor()创建张量，返回张量，数据类型，设备，梯度计算属性根据输入参数和默认值确定
labelTensor = torch.tensor(label, device=device)
tensorDataset = torch.utils.data.TensorDataset(featureTensor, labelTensor)
# TensorDataset()创建数据集，通常用于将多个张量组合成数据集，以便在训练时使用
# 返回数据集对象，该对象实现了__getitem__()和__len__()方法，使其可与数据加载器DataLoader一起使用
batch_size = 64
shuffle = True
dataloader = torch.utils.data.DataLoader(tensorDataset, batch_size, shuffle)
# DataLoader()从数据集中加载数据，并将其分批返回，通常与TensorDataset或其他自定义数据集类一起使用，以便在训练时高效加载和处理数据
# 返回数据加载器对象，该对象实现了__iter__()和__len__()方法，使其可在训练循环中使用

In [7]:
# 相关函数/方法


# torch.tensor(data, *, dtype=None, device=None, requires_grad=False)
# data输入数据，可是列表，元组，NumPy数组或其他可迭代对象
# dtype可选，指定张量数据类型，如果未指定，则根据输入数据自动推断数据类型
# device可选，指定张量所在的设备，如果未指定，则默认为CPU
# requires_grad可选，指定张量是否需要计算梯度，如果设为True，则张量会被标记为需要计算梯度，这对于反向传播和参数优化非常有用，默认为False
 
 
# torch.utils.data.TensorDataset(*tensors, transform=None)
# *tensors：一个或多个张量，这些张量将被组合成一个数据集，所有张量的第一个维度，通常是样本数量，必须相同
# transform可选，转换函数，在获取每个样本时对其进行转换
 
 
# torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None)
# dataset数据集对象，该对象必须实现__getitem__()和__len__()方法
# batch_size可选，批次大小，默认为1
# shuffle可选，如果设为True，则在每个epoch开始时打乱数据集的顺序，默认为False
# sampler可选，采样器对象，用于自定义数据的采样方式，如果提供，则shuffle参数将被忽略
# batch_sampler可选，批量采样器对象，用于自定义数据的批量采样方式，如果提供，则batch_size，shuffle，sampler，drop_last参数将被忽略
# num_workers可选，用于数据加载的子进程数，默认为0，表示在主进程中加载数据
# collate_fn可选，一个函数，用于将样本组合成批次，默认为None，表示使用默认的拼接方式
# pin_memory可选，如果设为True，则将数据加载到固定内存中，以便更快地将其传输到GPU，默认为False
# drop_last可选，如果设为True，则在最后一个批次中丢弃不足batch_size的样本，默认为False
# timeout可选，等待从数据加载器获取数据的超时时间，以秒为单位，默认为0，表示无限等待
# worker_init_fn可选，一个函数，用于初始化每个子进程，默认为None

### 定义模型

模型基于Embedding嵌入层，BiLSTM双向长短期记忆网络层，Dropout正则化层，全连接神经网络层。

1. Embedding(input_size, hidden_size)嵌入层，将输入的离散数据转为连续的向量表示，将输入序列x转为嵌入向量表示。

2. LSTM(hidden_size, hidden_size, bidirectional=True)双向长短期记忆网络层，捕捉输入序列的时序依赖关系，双向结构使模型能同时计算过去和未来的上下文信息，处理嵌入后的序列，捕捉上下文信息。

3. Dropout(dropout)正则化层，防止过拟合，通过随机丢弃一部分神经元来增加模型泛化能力。

4. Linear()全连接神经网络层，将输入数据映射到新的特征空间，从而提取更高层次的特征，增加模型复杂度，提高模型表达能力，执行分类任务。

5. ELU激活函数，缓解梯度消失问题，并引入非线性变换。

6. x, _ = self.bilstm(x)只取双向长短期记忆网络层的输出。

7. x = x[:, -1, :]只取最后一个时间步的输出，即模型计算的最后一个结果，即模型根据序列进行的预测。

注意：

- 通过多层全连接层和激活函数，以实现非线性变换，逐步提取高层次特征并最终输出预测结果。

- 模型最后的输出并没有使用类似Softmax激活函数功能的变换，因为CrossEntropyLoss()损失函数可自动实现，不必重复实现。

- 模型输入数据的形状均基于偶数设计，如果需要使用奇数，则需要手动调整部分模型参数。

- 没有手动初始化模型中网络的任何参数，均使用默认的自动初始化。

- 模型还可改造，还可添加其他正则化层，批量规范化层，其他需要的网络层等。

In [8]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, dropout, output_size):
        super().__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.bilstm = nn.LSTM(hidden_size, hidden_size, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size * 2, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, hidden_size // 4)
        self.fc3 = nn.Linear(hidden_size // 4, output_size)
 
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.bilstm(x)
        x = x[:, -1, :]
        x = self.dropout(x)
        x = self.fc1(x)
        x = nn.functional.elu(x)
        x = self.fc2(x)
        x = nn.functional.elu(x)
        x = self.fc3(x)
        return x

### 设置初始化模型所需的参数并初始化模型

1. 根据设备硬件情况，可将模型创建在指定设备上运行。

注意：

- 模型的输入和输出大小都由预处理得到的字典长度决定，代表处理的文本或字符串中的字符种类，便于分类。

- hidden_size并不只是指隐藏层的数量，还包括嵌入向量的大小，控制后续网络层的输入和输出大小。

- 控制Dropout正则化层的丢弃概率，较高的丢弃概率会增加正则化效果，但可能导致训练不稳定。

In [9]:
input_size = output_size = len(charToIndexVocabulary)
hidden_size = 32
dropout = 0.4
network = Net(input_size, hidden_size, dropout, output_size).to(device)
# to()将模型移动到指定设备

In [10]:
# 相关函数/方法


# tensor.to(device=None, *, non_blocking=False)
# device可选，指定目标设备
#   'cpu'：将张量移动到CPU
#   'cuda'：将张量移动到默认的GPU，通常是设备索引为0的GPU
#   'cuda:<device_index>'：将张量移动到指定索引的GPU
# non_blocking可选，如果设为True，则在移动张量时使用非阻塞传输，这可以在某些情况下提高性能，但需要确保目标设备的内存足够

### 定义损失函数和优化器并初始化所需参数

1. 损失函数使用CrossEntropyLoss()，用于分类问题的损失函数，结合了softmax层和负对数似然损失，衡量模型预测的概率分布与真实标签之间的差异。

2. 优化器使用Adam()，自适应学习率优化算法，结合了动量法和自适应学习率调整，能在训练模型时自动调整每个参数的学习率，有效加速收敛并提高模型性能。

注意：

- 可指定其他损失函数和优化器。

In [11]:
Loss = nn.CrossEntropyLoss()
# CrossEntropyLoss()交叉熵损失，通常用于多类分类问题，结合了LogSoftmax()和NLLLoss()，返回损失函数对象，可接受输入和目标张量
 
lr = 0.005
optimizer = torch.optim.Adam(network.parameters(), lr)
# Adam()实现Adam优化算法，自适应学习率优化算法，结合了动量和均方根传播，能在训练模型时自动调整每个参数的学习率，返回优化器对象，用于更新模型参数

In [12]:
# 相关函数/方法


# torch.nn.CrossEntropyLoss(weight=None, ignore_index=-100, reduction='mean')
# weight可选，一个一维张量，用于为每个类别指定权重，权重张量的长度应等于类别数量，如果某个类别的权重为0，则该类别的损失将被忽略
# ignore_index可选，整数，指定要忽略的目标值，在计算损失时，目标值为ignore_index的样本将被忽略
# reduction可选，指定损失的计算方式
#   none不进行任何缩减，返回每个样本的损失
#   mean默认，计算所有样本损失的平均值
#   sum计算所有样本损失的总和
 
 
# torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
# params可迭代对象，包含需要优化的参数
# lr学习率，默认为0.001
# betas一个包含两个浮点数的元组，分别表示动量衰减率和RMSprop衰减率，默认为(0.9, 0.999)
# eps一个很小的浮点数，用于防止除零错误，默认为1e-08
# weight_decay权重衰减率，用于实现L2正则化，默认为0
# amsgrad布尔值，表示是否使用AMSGrad变种算法，默认为False
# 学习率调整Adam，算法会自动调整每个参数的学习率，但在某些情况下，可能需要手动调整全局学习率
# 权重衰减，通过设置weight_decay参数，可实现L2正则化，有助于防止过拟合
# AMSGrad变种，设置amsgrad=True可以启用AMSGrad变种算法，该变种在某些情况下可以提供更好的性能

### 训练模型

1. 将模型转为训练模式，在训练模式下，某些特定的层会表现出与评估模式不同的行为。

2. 可指定训练轮数，使用训练轮数和数据集迭代，将特征输入到模型中，计算损失，清除上一次计算的梯度，根据损失计算梯度，更新模型参数。

3. 指定每50次打印一次损失值，每个批次结束后也打印。

In [13]:
network.train()
# train()定义和执行模型的训练过程
num_epochs = 2
for epoch in range(num_epochs):
    i = 1
    for features, label in dataloader:
        output = network(features)
        loss = Loss(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        if i % 50 == 0:
            print(f'第{epoch + 1}轮，第{i}个损失：{loss}')
    print(f'第{epoch + 1}轮最后一次损失：{loss}')

第1轮，第50个损失：2.9421072006225586
第1轮，第100个损失：2.8394737243652344
第1轮，第150个损失：2.842772960662842
第1轮，第200个损失：2.7504196166992188
第1轮，第250个损失：2.6078498363494873
第1轮，第300个损失：2.70068097114563
第1轮，第350个损失：2.63191819190979
第1轮，第400个损失：2.5804219245910645
第1轮最后一次损失：2.7951714992523193
第2轮，第50个损失：2.3706142902374268
第2轮，第100个损失：2.6182005405426025
第2轮，第150个损失：2.561777353286743
第2轮，第200个损失：2.5700271129608154
第2轮，第250个损失：2.6232640743255615
第2轮，第300个损失：2.631828546524048
第2轮，第350个损失：2.7216098308563232
第2轮，第400个损失：2.6682698726654053
第2轮最后一次损失：2.7489914894104004


### 模型预测

1. 将模型转为评估模式，关闭训练时特有的层，确保在预测数据上获得一致且可重复的结果。

2. 使用字符串进行预测，指定预处理字符串的截取窗口大小和预测步数/长度，预处理字符串。

3. 禁用梯度计算，防止模型内部参数变化，通过预测步数/长度和循环指定模型运行次数。

4. 根据截取窗口大小截取字符串的最后一部分，转为数字序列，再转为张量，放入模型得到输出，用argmax()取最大的数值的索引，即预测结果，然后添加到数字序列中。

5. 将数字序列转为字符序列并打印出整个字符串。

注意：

- 也可使用文本进行预测，根据需求需要使用相应预处理方法。

In [14]:
network.eval()
# eval()模型方法，将模型设置为评估模式，在评估模式下，模型会关闭训练时特有的层，以确保在评估或测试数据上获得一致且可重复的结果
# 在评估模式下，通常会使用torch.no_grad()上下文管理器来禁用梯度计算，可减少内存消耗，加快计算速度，因为在评估过程中不需要更新模型参数
window = 5
num_steps = 100
string = 'LiLing_KongBai'
string = YuChuLi.stringYuChuLi(string)
stringIndexSequence = YuChuLi.charToIndexTransform(string, charToIndexVocabulary)
with torch.no_grad():
    for i in range(num_steps):
        example = torch.tensor(stringIndexSequence[-window :]).unsqueeze(0).to(device)
        new_output = network(example)
        predict_index = new_output.argmax(dim=1, keepdim=True)
        stringIndexSequence.append(predict_index.item())
predict_char = YuChuLi.indexToCharTransform(stringIndexSequence, indexToCharVocabulary)
predict_char = ''.join(predict_char)
print(predict_char) 

liling kongbai o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o


### 手动单步预测

注意：

- 这是将预测部分的代码提取，用于手动运行单步预测。

- 实现每运行一次，生成一个字符，然后查看原字符串和预测结果拼接的完整字符串，便于观测预测结果，打印每步输出，查看各步输入输出的变化。

- 需要自行查看预测的结果，让后手动将srting2的值更改。

In [15]:
string2 = 'LiLing_KongBai'
string2 = YuChuLi.stringYuChuLi(string2)
stringIndexSequence2 = YuChuLi.charToIndexTransform(string2, charToIndexVocabulary)
example2 = torch.tensor(stringIndexSequence2[-window :]).unsqueeze(0).to(device)
output2 = network(example2)
predict_index2 = output2.argmax(dim=1, keepdim=True)
stringIndexSequence2.append(predict_index2.item())
predict_char2 = YuChuLi.indexToCharTransform(stringIndexSequence2, indexToCharVocabulary)
predict_char2 = ''.join(predict_char2)
print(predict_char2)

liling kongbai 
